In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName("logistic-regression").getOrCreate()


In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [11]:
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [ ]:
# text칼럼에 들어있는 spark 단어가 있으면 label이 1.0이 된다.

In [12]:
# 글자들을 split 해주기 위해 Tokenizer 사용
tokenizer = Tokenizer(inputCol="text",outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

In [13]:
lr = LogisticRegression(maxIter=30, regParam=0.001)

In [14]:
# pipeline 만들기

pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [15]:
model = pipeline.fit(training)

22/04/06 17:22:27 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/06 17:22:27 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/04/06 17:22:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/04/06 17:22:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [16]:
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [17]:
prediction = model.transform(test)

In [18]:
prediction.show()

+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|              text|               words|            features|       rawPrediction|         probability|prediction|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4|       spark i j k|    [spark, i, j, k]|(262144,[19036,68...|[0.53662516439882...|[0.63102699631690...|       0.0|
|  5|             l m n|           [l, m, n]|(262144,[1303,526...|[4.17742695597525...|[0.98489377609773...|       0.0|
|  6|spark hadoop spark|[spark, hadoop, s...|(262144,[173558,1...|[-1.8520577251150...|[0.13563147748816...|       1.0|
|  7|     apache hadoop|    [apache, hadoop]|(262144,[68303,19...|[5.42954585803784...|[0.99563405823116...|       0.0|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+



In [19]:
prediction.select(['id', 'text', 'probability', 'prediction']).show()

+---+------------------+--------------------+----------+
| id|              text|         probability|prediction|
+---+------------------+--------------------+----------+
|  4|       spark i j k|[0.63102699631690...|       0.0|
|  5|             l m n|[0.98489377609773...|       0.0|
|  6|spark hadoop spark|[0.13563147748816...|       1.0|
|  7|     apache hadoop|[0.99563405823116...|       0.0|
+---+------------------+--------------------+----------+

